In [1]:
import sys
import stlearn as st
st.settings.set_figure_params(dpi=300)
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import sys
file = Path("../stimage").resolve()
parent= file.parent
sys.path.append(str(parent))
from PIL import Image
from stimage._utils import gene_plot, Read10X, ReadOldST, tiling, ensembl_to_id
from stimage._model import CNN_NB_multiple_genes, negative_binomial_layer, negative_binomial_loss
from stimage._data_generator import DataGenerator
import tensorflow as tf
import seaborn as sns
sns.set_style("white")
import matplotlib.pyplot as plt
from scipy import stats
import numpy as np
# import geopandas as gpd
from sklearn.neighbors import KDTree
from anndata import read_h5ad
from tensorflow.keras import backend as K
import scanpy as sc

In [2]:
import matplotlib.pyplot as plt
from libpysal.weights.contiguity import Queen
from libpysal import examples
import numpy as np
import pandas as pd
import geopandas as gpd
import os
import splot
from splot.esda import moran_scatterplot, lisa_cluster
from esda.moran import Moran, Moran_Local
from esda.moran import Moran_BV, Moran_Local_BV
from splot.esda import plot_moran_bv_simulation, plot_moran_bv, plot_local_autocorrelation

In [3]:
from scipy import stats

def plot_correlation(df, attr_1, attr_2):
    r = stats.pearsonr(df[attr_1], 
                       df[attr_2])[0] **2

    g = sns.lmplot(data=df,
        x=attr_1, y=attr_2,
        height=5, legend=True
    )
    # g.set(ylim=(0, 360), xlim=(0,360))

    g.set_axis_labels(attr_1, attr_2)
    plt.annotate(r'$R^2:{0:.2f}$'.format(r),
                (max(df[attr_1])*0.9, max(df[attr_2])*0.9))
    return g


def calculate_correlation(attr_1, attr_2):
    r = stats.pearsonr(attr_1, 
                       attr_2)[0]
    return r

def calculate_correlation_2(attr_1, attr_2):
    r = stats.spearmanr(attr_1, 
                       attr_2)[0]
    return r

In [4]:
DATA_PATH = Path("/clusterdata/uqxtan9/Q1851/Xiao/Working_project/her2st")
adata_all = read_h5ad(DATA_PATH / "all_adata_no_normal_224.h5ad")

In [5]:
samples = adata_all.obs["library_id"].unique().tolist()

In [28]:
gene_list=["COX6C","TTLL12", "PABPC1", "GNAS", "HSP90AB1", 
           "TFF3", "ATP1A1", "B2M", "FASN", "SPARC", "CD74", "CD63", "CD24", "CD81"]

In [29]:
test_sample = samples[0]
n_genes = len(gene_list)

adata_all_train_valid = adata_all[adata_all.obs["library_id"].isin(
    adata_all.obs.library_id.cat.remove_categories(test_sample).unique())]

training_index = adata_all_train_valid.obs.sample(frac=0.7, random_state=1).index
training_dataset = adata_all_train_valid[training_index,].copy()

valid_index = adata_all_train_valid.obs.index.isin(training_index)
valid_dataset = adata_all_train_valid[~valid_index,].copy()

test_index = adata_all.obs.library_id == test_sample
test_dataset_1 = adata_all[test_index,].copy()

In [34]:
train_gen = tf.data.Dataset.from_generator(
            lambda:DataGenerator(adata=training_dataset, 
                          genes=gene_list, aug=False, dim=(224, 224)),
            output_types=(tf.float32, tuple([tf.float32]*n_genes)), 
            output_shapes=([224,224,3], tuple([1]*n_genes))
)
train_gen_ = train_gen.shuffle(buffer_size=500).batch(32).repeat(1).cache().prefetch(tf.data.experimental.AUTOTUNE)
valid_gen = tf.data.Dataset.from_generator(
            lambda:DataGenerator(adata=valid_dataset, 
                          genes=gene_list, dim=(224, 224)), 
            output_types=(tf.float32, tuple([tf.float32]*n_genes)), 
            output_shapes=([224,224,3], tuple([1]*n_genes))
)
valid_gen_ = valid_gen.shuffle(buffer_size=500).batch(32).repeat(1).cache().prefetch(tf.data.experimental.AUTOTUNE)
test_gen_1 = tf.data.Dataset.from_generator(
            lambda:DataGenerator(adata=test_dataset_1, 
                          genes=gene_list, dim=(224, 224)), 
            output_types=(tf.float32, tuple([tf.float32]*n_genes)), 
            output_shapes=([224,224,3], tuple([1]*n_genes))
)
test_gen__1 = test_gen_1.batch(1)

In [50]:
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input as preprocess_vgg16
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input as preprocess_resnet50
from tensorflow.keras.applications.densenet import DenseNet121, preprocess_input as preprocess_densenet
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Input, concatenate, Dropout, Lambda
from tensorflow.keras.models import Model
from tensorflow.python.keras import backend as K
from tensorflow.python.keras.utils.tf_utils import is_tensor_or_variable


def STNet(tile_shape, output_shape, mean_exp_tf):
    tile_input = Input(shape=tile_shape, name = "tile_input")
    DenseNet121_base = DenseNet121(input_tensor=tile_input, weights='imagenet', include_top=False)
    for layer in DenseNet121_base.layers:
        layer.trainable = False
    
    cnn = DenseNet121_base.output
    cnn = GlobalAveragePooling2D()(cnn)
#     cnn = Dropout(0.5)(cnn)
#     cnn = Dense(256, activation='relu', kernel_regularizer=tf.keras.regularizers.l1(0.01),
#                 activity_regularizer=tf.keras.regularizers.l2(0.01))(cnn)
    # cnn = Dense(256, activation='relu')(cnn)
    
    outputs = Dense(output_shape, activation='linear', bias_initializer=mean_exp_tf)(cnn)
    model = Model(inputs=tile_input, outputs=outputs)

#     optimizer = tf.keras.optimizers.RMSprop(0.001)
    optimizer = tf.keras.optimizers.SGD(learning_rate=1e-6, momentum=0.9)

    model.compile(loss="mse",
                  optimizer=optimizer,
                  metrics=[tf.keras.metrics.MeanSquaredError()])    
    return model

In [53]:
mean_exp = training_dataset[:,gene_list].to_df().mean()
mean_exp_tf = tf.keras.initializers.RandomUniform(minval=mean_exp, 
                                                  maxval=mean_exp)

In [54]:
model = STNet((224, 224, 3), n_genes, mean_exp_tf)

In [55]:
K.clear_session()
train_history = model.fit(train_gen_,
                          epochs=1,
                          validation_data=valid_gen_)

294/294 [==============================] - 23s 67ms/step - loss: 27.2089 - mean_squared_error: 27.2089 - val_loss: 5.2847 - val_mean_squared_error: 5.2847


In [56]:
test_predictions = model.predict(test_gen__1)

In [57]:
test_dataset_1.obsm["predicted_gene"] = test_predictions
test_dataset_1_ = test_dataset_1[:,gene_list].copy()
test_dataset_1_.X = test_dataset_1_.obsm["predicted_gene"]

In [60]:
pred_adata = test_dataset_1_
test_dataset = test_dataset_1

for gene in pred_adata.var_names:
    cor_val = calculate_correlation(pred_adata.to_df().loc[:,gene], test_dataset.to_df().loc[:,gene])
    df = df.append(pd.Series([gene, cor_val, test_sample, "STimage"], 
                         index=["Gene", "Pearson correlation", "Slide", "Method"]),
                  ignore_index=True)

In [61]:
df

,Gene,Pearson correlation,Slide,Method
0,COX6C,-0.332004,C2,STimage
1,TTLL12,-0.070448,C2,STimage
2,PABPC1,0.109870,C2,STimage
3,GNAS,0.312379,C2,STimage
4,HSP90AB1,0.112822,C2,STimage
5,TFF3,-0.071778,C2,STimage
6,ATP1A1,0.000877,C2,STimage
7,B2M,0.132214,C2,STimage
8,FASN,-0.204973,C2,STimage
9,SPARC,0.011849,C2,STimage


In [59]:
df=pd.DataFrame()